<img src="img/LogoWekeo_Copernicus_RGB_0.png" alt="" width="12%" align="left">

# How to use the WEkEO Harmonized Data Access (HDA) API?

[WEkEO](https://wekeo.copernicus.eu/) is the EU Copernicus DIAS (Data and Information Access Service) reference service for environmental data, virtual processing environments and skilled user support. WEkEO offers access to a variety of data, including all Copernicus services and a multitude of parameters sensored from Sentinel-1, Sentinel-2,  Sentinel-3, Sentinel-5P and SPOT satellites family. It further offers access to climate reanalysis and seasonal forecast data.

The [Harmonised Data Access (HDA) API](https://help.wekeo.eu/en/articles/9515753-what-is-the-harmonized-data-access-hda-api), a REST interface, allows users to subset and download datasets from WEkEO.

This notebook is a step-by-step guide on how to search for and download data from WEkEO using the `HDA API`, following these steps:

 - [1. Import the WEkEO HDA client](#import_hda)
 - [2. Configure the HDA API Authentication](#hda_auth)
 - [3. Search for datasets on WEkEO](#dataset_search)
 - [4. Create the data request](#create_request)
 - [5. Download data](#download_data)

### <a id='import_hda'></a>1. Import the WEkEO HDA client

The `hda` package provides a fully compliant Python 3 client that can be used to search and download products using the WEkEO Harmonized Data Access API.  
The HDA is a RESTful interface allowing users to search and download WEkEO datasets.
For more information, please check the [official documentation](https://hda.readthedocs.io/en/latest/index.html) and our [Help Center](https://help.wekeo.eu/en/collections/3530725-wekeo-data-download).

If you are working in the WEkEO JupyterLab, the **HDA client is already installed in the `wekeolab` kernel**, so make sure it is selected on the top right and simply run the following cell:

In [2]:
from hda import Client, Configuration

*Note: if you're working in a local environment, you can install the `hda` package easily via PyPI or Mamba/Conda as shown [in this article](https://help.wekeo.eu/en/articles/6751608-how-to-use-the-hda-api-in-python#h_a309ecb279).*

### <a id='hda_auth'></a>2. Configure the HDA API Authentication

In order to download data using the HDA API, you need to provide your WEkEO credentials. You can do this in two ways:

* **Option 1**: by creating a credentials file (*recommended, to be done only once*)
* **Option 2**: by supplying your credentials directly in the script (*not recommended, but sometimes useful*)

#### **Option 1: creating a credentials file (only once)**

By default, the HDA API expects the configuration to be called `.hdarc`, and to reside in your home directory.  
The easiest way to create the credentials file is to run the following cell and enter your WEkEO username and password:  

In [ ]:
from pathlib import Path

# Default location expected by hda package
hdarc = Path(Path.home() / '.hdarc')

# Create it only if it does not already exists
if not hdarc.is_file():
    import getpass
    USERNAME = input('Enter your username: ')
    PASSWORD = getpass.getpass('Enter your password: ')

    with open(Path.home() / '.hdarc', 'w') as f:
        f.write(f'user:{USERNAME}\n')
        f.write(f'password:{PASSWORD}\n')

*Note: if you don't have a WEkEO account yet, you can [register for free!](https://wekeo.copernicus.eu/register)*

Now the HDA will recognize your WEkEO credentials automatically, and you can simply call the HDA client as follows:

In [2]:
hda_client = Client()

#### **Option 2: providing credentials directly (every time)**

You can provide your credentials directly as follows, replacing both username and password:

In [ ]:
conf = Configuration(user="username", password="password")
hda_client = Client(config=conf)

*Note: this method is convenient in the short term, but is not really recommended as you have to put your username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*

### <a id='dataset_search'></a>3. Search for datasets on WEkEO

Once on the WEkEO [Expert Data Viewer](https://wekeo.copernicus.eu/data?view=viewer), click on the "+" icon from the Layers tab to open the Catalogue and search for your dataset of interest. You can use the free-text search bar (e.g. *`sentinel-3`*) and browse through the results. You can also use the filters on the left to refine your search and look by satellite plaform, sensor, Copernicus service, area (region of interest), as well as through a variety of flags.

<img src='./img/WEkEO_catalogue.png' width='80%'></img>

You can click on **Details** for the dataset you are interested in to open the dataset's page, including a description, temporal and spatial extents, and metadata.  
For the purpose of this notebook, we will take the dataset `OLCI Level 1B Full Resolution - Sentinel-3`:

<img src='./img/Sentinel_3_datasetID.png' width='50%'></img>

The dataset's page provides the following information:
- **Abstract**: containing a general description of the dataset
- **Classification**: including the datasetID 
- **Resources**: such as a link to the Product Data Format Specification guide, and JSON metadata
- **Contacts**: where you can find further information about the data source from its provider

Click on **Add to map** to add the dataset to the Layers and visualize data on the map.  

### <a id='create_request'></a>4. Create the data request

The HDA API reads the data query from a JSON-like dictionary. Depending on the selected dataset, certain parameters are mandatory.  
To avoid mistakes, it is **highly recommended to build the query from the [Expert Data Viewer](https://wekeo.copernicus.eu/data?view=viewer)**.

From the Layers tab, you can click on the **Subset and download** button to build the data request via Graphical User Interface:

<img src='./img/download_button.png' width='20%'></img>

Once you defined the download parameters, you can `(1)` click on **Show API request(s)** and `(2)` on **Copy**:

<img src='./img/api_request.png' width='60%' />


Then you can paste the request in a variable as follows:

In [3]:
query = {
  "dataset_id": "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___",
  "startdate": "2025-05-12T00:00:00.000Z",
  "enddate": "2025-05-14T23:59:59.999Z",
  "bbox": [
    1.3287978665692197,
    45.538598473683436,
    5.158722327438545,
    47.78018177390331
  ],
  "type": "OL_1_EFR___",
  "timeliness": "NT",
  "itemsPerPage": 200,
  "startIndex": 0
}

The following cell is used to perform the search based on these parameters, and the results are stored in the `matches` variable:

In [ ]:
# Ask the result for the query passed in parameter
matches = hda_client.search(query)

# List the results
print(matches)

### <a id='download_data'></a>6. Download data

As a final step, you simply need to download the results of the `matches` variable:

In [6]:
matches.download()

*Note: you can check [our article](https://help.wekeo.eu/en/articles/6751608-how-to-use-the-hda-api-in-python#h_80a0396d92) to download specific files of the results.*

***

<a href="https://github.com/wekeo/wekeo4data" target="_blank">View on GitHub</a> | [Contact the WEkEO User Support](https://wekeo.copernicus.eu/#help-support) | [WEkEO Help Center](https://help.wekeo.eu/en/)